# Atividade 2 - Avaliação e Métricas do Modelo XGBoost

**Curso:** Qualificação em IA Industrial  
**Unidade Curricular:** Machine Learning  
**Objetivo:** Avaliar o desempenho de um modelo XGBoost para predição de atrasos de voos

## Objetivo da Atividade
Avaliar o desempenho de um modelo XGBoost utilizando métricas avançadas de classificação, incluindo matriz de confusão, curva ROC e AUC.

## Contexto do Problema
Um grande site de reservas de viagens quer criar um recurso que informe a probabilidade de atraso do voo no momento da reserva, usando dados históricos de voos.

### Etapas da Atividade:
1. **Importar e preparar os dados**
2. **Treinar o modelo XGBoost**
3. **Predição do modelo (probabilidades e classes)**
4. **Matriz de Confusão**
5. **Métricas de desempenho detalhadas**
6. **Curva ROC e AUC**

## 1. Importação das Bibliotecas

Importando todas as bibliotecas necessárias para análise, modelagem e visualização:

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    roc_curve, auc, roc_auc_score, precision_score, recall_score,
    f1_score, precision_recall_curve
)
from xgboost import XGBClassifier

# Configurações para visualização
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

print("Todas as bibliotecas importadas com sucesso!")

## 2. Carregamento e Preparação dos Dados

Carregando o dataset e preparando os dados para o modelo:

In [ ]:
# Carregamento dos dados
df = pd.read_csv('flights_delays_120.csv')

print("INFORMAÇÕES DO DATASET")
print(f"Dimensões: {df.shape}")
print(f"Colunas: {list(df.columns)}")

# Visualizando as primeiras linhas
print("\nPrimeiras 5 linhas do dataset:")
display(df.head())

# Informações sobre os dados
print("\nInformações gerais:")
print(df.info())

# Verificando valores ausentes
print(f"\nValores ausentes:")
print(df.isnull().sum())

In [ ]:
# Análise da variável target
print("ANÁLISE DA VARIÁVEL TARGET (delayed)")
print(f"Distribuição da variável 'delayed':")
target_counts = df['delayed'].value_counts()
target_prop = df['delayed'].value_counts(normalize=True)

for i, (count, prop) in enumerate(zip(target_counts, target_prop)):
    label = "Não Atrasado" if i == 0 else "Atrasado"
    print(f"  {label}: {count} ({prop:.2%})")

# Visualização da distribuição da variável target
plt.figure(figsize=(12, 5))

# Gráfico de barras - contagem
plt.subplot(1, 2, 1)
target_counts.plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Distribuição de Atrasos - Contagem')
plt.xlabel('Delayed')
plt.ylabel('Quantidade')
plt.xticks([0, 1], ['Não Atrasado', 'Atrasado'], rotation=0)

# Gráfico de pizza - proporção
plt.subplot(1, 2, 2)
plt.pie(target_counts.values, labels=['Não Atrasado', 'Atrasado'], 
        autopct='%1.1f%%', colors=['skyblue', 'salmon'])
plt.title('Proporção de Atrasos')

plt.tight_layout()
plt.show()

In [ ]:
# Preparação dos dados para o modelo
print("PREPARAÇÃO DOS DADOS")

# Separação de features (X) e target (y)
X = df.drop('delayed', axis=1)
y = df['delayed']

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Tratamento de variáveis categóricas
print(f"\nVariáveis categóricas encontradas: {X.select_dtypes(include=['object']).columns.tolist()}")

# Convertendo variáveis categóricas em numéricas usando get_dummies
X_encoded = pd.get_dummies(X, drop_first=True)

print(f"Dimensões após encoding: {X_encoded.shape}")
print(f"Novas features criadas: {X_encoded.shape[1] - X.shape[1]}")

# Verificando se há colunas numéricas que precisam de normalização
print(f"\nTipos de dados após encoding:")
print(X_encoded.dtypes.value_counts())

print(f"\nPrimeiras colunas após encoding:")
print(list(X_encoded.columns[:10]))

In [ ]:
# Divisão dos dados em treino e teste com estratificação
print("DIVISÃO DOS DADOS")

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Garantindo estratificação
)

print(f"Conjunto de treinamento: {X_train.shape}")
print(f"Conjunto de teste: {X_test.shape}")

# Verificando a estratificação
print(f"\nDistribuição no conjunto de TREINAMENTO:")
train_dist = y_train.value_counts(normalize=True)
for i, prop in enumerate(train_dist):
    label = "Não Atrasado" if i == 0 else "Atrasado"
    print(f"  {label}: {prop:.2%}")

print(f"\nDistribuição no conjunto de TESTE:")
test_dist = y_test.value_counts(normalize=True)
for i, prop in enumerate(test_dist):
    label = "Não Atrasado" if i == 0 else "Atrasado"
    print(f"  {label}: {prop:.2%}")

print(f"\nEstratificação mantida com sucesso!")

## 3. Treinamento do Modelo XGBoost

Criando e treinando o modelo XGBoost com parâmetros otimizados:

In [ ]:
# Definição e treinamento do modelo XGBoost
print("TREINAMENTO DO MODELO XGBOOST")

# Definindo o classificador XGBoost com parâmetros simples
model = XGBClassifier(
    random_state=42,
    eval_metric='logloss',  # Métrica de avaliação
    n_estimators=100,       # Número de árvores
    max_depth=6,            # Profundidade máxima das árvores
    learning_rate=0.1       # Taxa de aprendizado
)

print(f"Modelo configurado com os seguintes parâmetros:")
for param, value in model.get_params().items():
    if param in ['random_state', 'eval_metric', 'n_estimators', 'max_depth', 'learning_rate']:
        print(f"  {param}: {value}")

# Treinamento do modelo
print(f"\nIniciando treinamento...")
model.fit(X_train, y_train)
print(f"Treinamento concluído com sucesso!")

# Verificando informações do modelo treinado
print(f"\nInformações do modelo treinado:")
print(f"Número de features: {model.n_features_in_}")
print(f"Número de classes: {len(model.classes_)}")
print(f"Classes: {model.classes_}")

## 4. Predições do Modelo

Realizando predições com probabilidades e conversão para classes:

In [ ]:
# Realizando predições com probabilidades
print("PREDIÇÕES DO MODELO")

# Obtendo probabilidades para cada classe
y_proba = model.predict_proba(X_test)
print(f"Forma das probabilidades: {y_proba.shape}")
print(f"Colunas: [Prob_Não_Atrasado, Prob_Atrasado]")

# Probabilidades para a classe positiva (atrasado = 1)
y_proba_positive = y_proba[:, 1]

print(f"\nEstatísticas das probabilidades (classe positiva):")
print(f"Mínima: {y_proba_positive.min():.4f}")
print(f"Máxima: {y_proba_positive.max():.4f}")
print(f"Média: {y_proba_positive.mean():.4f}")
print(f"Mediana: {np.median(y_proba_positive):.4f}")

# Convertendo probabilidades em classes usando threshold 0.5
threshold = 0.5
y_pred = (y_proba_positive >= threshold).astype(int)

print(f"\nConversão para classes (threshold = {threshold}):")
print(f"Predições únicas: {np.unique(y_pred)}")
print(f"Distribuição das predições:")
pred_counts = pd.Series(y_pred).value_counts()
for i, count in enumerate(pred_counts):
    label = "Não Atrasado" if i == 0 else "Atrasado"
    print(f"  {label}: {count}")

# Exemplos de predições
print(f"\nPrimeiras 10 predições:")
results_df = pd.DataFrame({
    'Real': y_test.iloc[:10].values,
    'Prob_Atrasado': y_proba_positive[:10],
    'Predição': y_pred[:10]
})
display(results_df)

## 5. Matriz de Confusão

Calculando e visualizando a matriz de confusão:

In [ ]:
# Calculando a matriz de confusão
print("MATRIZ DE CONFUSÃO")

cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(cm)

# Extraindo os valores TP, TN, FP, FN
tn, fp, fn, tp = cm.ravel()

print(f"\nComponentes da Matriz de Confusão:")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

# Visualização da matriz de confusão
plt.figure(figsize=(15, 6))

# Matriz de confusão simples
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Não Atrasado', 'Atrasado'],
            yticklabels=['Não Atrasado', 'Atrasado'])
plt.title('Matriz de Confusão - Valores Absolutos')
plt.xlabel('Predição')
plt.ylabel('Real')

# Matriz de confusão normalizada
plt.subplot(1, 2, 2)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=['Não Atrasado', 'Atrasado'],
            yticklabels=['Não Atrasado', 'Atrasado'])
plt.title('Matriz de Confusão - Normalizada')
plt.xlabel('Predição')
plt.ylabel('Real')

plt.tight_layout()
plt.show()

# Interpretação detalhada
print(f"\nINTERPRETAÇÃO:")
print(f"• TN ({tn}): Voos corretamente identificados como NÃO atrasados")
print(f"• FP ({fp}): Voos incorretamente identificados como atrasados (Erro Tipo I)")
print(f"• FN ({fn}): Voos incorretamente identificados como NÃO atrasados (Erro Tipo II)")
print(f"• TP ({tp}): Voos corretamente identificados como atrasados")

## 6. Métricas de Desempenho Detalhadas

Calculando todas as métricas importantes para avaliação do modelo:

In [ ]:
# Calculando métricas de desempenho detalhadas
print("MÉTRICAS DE DESEMPENHO")

# Métricas calculadas manualmente a partir da matriz de confusão
accuracy_manual = (tp + tn) / (tp + tn + fp + fn)
sensitivity_recall = tp / (tp + fn)  # Sensibilidade ou Recall
specificity = tn / (tn + fp)         # Especificidade
precision = tp / (tp + fp)           # Precisão
fpr = fp / (fp + tn)                 # Taxa de Falsos Positivos
fnr = fn / (fn + tp)                 # Taxa de Falsos Negativos
f1 = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall)

print(f"MÉTRICAS CALCULADAS MANUALMENTE:")
print(f"Acurácia: {accuracy_manual:.4f} ({accuracy_manual*100:.2f}%)")
print(f"Sensibilidade (Recall): {sensitivity_recall:.4f} ({sensitivity_recall*100:.2f}%)")
print(f"Especificidade: {specificity:.4f} ({specificity*100:.2f}%)")
print(f"Precisão: {precision:.4f} ({precision*100:.2f}%)")
print(f"F1-Score: {f1:.4f}")
print(f"Taxa de Falsos Positivos (FPR): {fpr:.4f} ({fpr*100:.2f}%)")
print(f"Taxa de Falsos Negativos (FNR): {fnr:.4f} ({fnr*100:.2f}%)")

# Verificação com funções do sklearn
print(f"\nVERIFICAÇÃO COM SKLEARN:")
accuracy_sklearn = accuracy_score(y_test, y_pred)
precision_sklearn = precision_score(y_test, y_pred)
recall_sklearn = recall_score(y_test, y_pred)
f1_sklearn = f1_score(y_test, y_pred)

print(f"Acurácia (sklearn): {accuracy_sklearn:.4f}")
print(f"Precisão (sklearn): {precision_sklearn:.4f}")
print(f"Recall (sklearn): {recall_sklearn:.4f}")
print(f"F1-Score (sklearn): {f1_sklearn:.4f}")

# Relatório de classificação completo
print(f"\nRELATÓRIO DE CLASSIFICAÇÃO:")
print(classification_report(y_test, y_pred, target_names=['Não Atrasado', 'Atrasado']))

In [ ]:
# Visualização das métricas
print("VISUALIZAÇÃO DAS MÉTRICAS")

# Criando um DataFrame com as métricas
metrics_data = {
    'Métrica': ['Acurácia', 'Sensibilidade\n(Recall)', 'Especificidade', 
                'Precisão', 'F1-Score', 'FPR', 'FNR'],
    'Valor': [accuracy_manual, sensitivity_recall, specificity, 
              precision, f1, fpr, fnr],
    'Interpretação': [
        'Proporção de acertos totais',
        'Capacidade de identificar atrasos',
        'Capacidade de identificar não-atrasos',
        'Confiabilidade das predições positivas',
        'Harmonia entre precisão e recall',
        'Taxa de alarmes falsos',
        'Taxa de atrasos perdidos'
    ]
}

metrics_df = pd.DataFrame(metrics_data)

# Gráfico de barras das métricas principais
plt.figure(figsize=(15, 10))

# Gráfico das métricas principais
plt.subplot(2, 2, 1)
main_metrics = ['Acurácia', 'Sensibilidade\n(Recall)', 'Especificidade', 'Precisão', 'F1-Score']
main_values = [accuracy_manual, sensitivity_recall, specificity, precision, f1]
colors = ['skyblue', 'lightgreen', 'salmon', 'gold', 'lightcoral']

bars = plt.bar(main_metrics, main_values, color=colors)
plt.title('Métricas Principais do Modelo')
plt.ylabel('Valor')
plt.ylim(0, 1)

# Adicionando valores nas barras
for bar, value in zip(bars, main_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.3f}', ha='center', va='bottom')

plt.xticks(rotation=45)

# Gráfico das taxas de erro
plt.subplot(2, 2, 2)
error_metrics = ['FPR', 'FNR']
error_values = [fpr, fnr]
error_colors = ['orange', 'red']

bars2 = plt.bar(error_metrics, error_values, color=error_colors)
plt.title('Taxas de Erro')
plt.ylabel('Taxa')
plt.ylim(0, max(error_values) * 1.2)

for bar, value in zip(bars2, error_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.3f}', ha='center', va='bottom')

# Comparação das métricas em formato radar (simplificado)
plt.subplot(2, 2, 3)
metrics_comparison = pd.DataFrame({
    'Métrica': ['Acurácia', 'Precisão', 'Recall', 'F1-Score'],
    'Valor': [accuracy_manual, precision, sensitivity_recall, f1]
})

plt.plot(metrics_comparison['Métrica'], metrics_comparison['Valor'], 
         marker='o', linewidth=2, markersize=8)
plt.title('Comparação de Métricas')
plt.ylabel('Valor')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Tabela resumo
plt.subplot(2, 2, 4)
plt.axis('off')
table_data = []
for _, row in metrics_df.iterrows():
    table_data.append([row['Métrica'], f"{row['Valor']:.4f}", row['Interpretação']])

table = plt.table(cellText=table_data,
                  colLabels=['Métrica', 'Valor', 'Interpretação'],
                  cellLoc='left',
                  loc='center')
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)
plt.title('Resumo das Métricas')

plt.tight_layout()
plt.show()

print(f"\nINTERPRETAÇÃO DAS MÉTRICAS:")
print(f"• Acurácia ({accuracy_manual:.1%}): {accuracy_manual*100:.1f}% dos casos foram classificados corretamente")
print(f"• Sensibilidade ({sensitivity_recall:.1%}): {sensitivity_recall*100:.1f}% dos atrasos reais foram identificados")
print(f"• Especificidade ({specificity:.1%}): {specificity*100:.1f}% dos não-atrasos reais foram identificados")
print(f"• Precisão ({precision:.1%}): {precision*100:.1f}% das predições de atraso foram corretas")

## 7. Curva ROC e AUC

Gerando a curva ROC e calculando a área sob a curva (AUC):

In [ ]:
# Calculando a Curva ROC e AUC
print("CURVA ROC E AUC")

# Calculando os pontos da curva ROC
fpr_roc, tpr_roc, thresholds = roc_curve(y_test, y_proba_positive)

# Calculando a AUC
auc_score = auc(fpr_roc, tpr_roc)
auc_sklearn = roc_auc_score(y_test, y_proba_positive)

print(f"AUC calculada manualmente: {auc_score:.4f}")
print(f"AUC calculada com sklearn: {auc_sklearn:.4f}")

# Visualização da Curva ROC
plt.figure(figsize=(15, 10))

# Curva ROC principal
plt.subplot(2, 2, 1)
plt.plot(fpr_roc, tpr_roc, linewidth=2, label=f'ROC Curve (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC (Receiver Operating Characteristic)')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# Adicionando o ponto do nosso modelo com threshold 0.5
current_fpr = fp / (fp + tn)
current_tpr = tp / (tp + fn)
plt.plot(current_fpr, current_tpr, 'ro', markersize=8, 
         label=f'Modelo Atual (threshold=0.5)')
plt.legend(loc="lower right")

# Zoom na região interessante da curva ROC
plt.subplot(2, 2, 2)
plt.plot(fpr_roc, tpr_roc, linewidth=2, label=f'ROC Curve (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlim([0.0, 0.4])  # Zoom na parte interessante
plt.ylim([0.6, 1.0])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC - Zoom')
plt.plot(current_fpr, current_tpr, 'ro', markersize=8)
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# Distribuição das probabilidades
plt.subplot(2, 2, 3)
plt.hist(y_proba_positive[y_test == 0], bins=50, alpha=0.7, 
         label='Não Atrasado', color='skyblue', density=True)
plt.hist(y_proba_positive[y_test == 1], bins=50, alpha=0.7, 
         label='Atrasado', color='salmon', density=True)
plt.axvline(x=threshold, color='red', linestyle='--', linewidth=2, 
           label=f'Threshold = {threshold}')
plt.xlabel('Probabilidade Predita')
plt.ylabel('Densidade')
plt.title('Distribuição das Probabilidades')
plt.legend()
plt.grid(True, alpha=0.3)

# Análise de diferentes thresholds
plt.subplot(2, 2, 4)
thresholds_analysis = np.linspace(0, 1, 101)
precision_list = []
recall_list = []
f1_list = []

for thresh in thresholds_analysis:
    y_pred_thresh = (y_proba_positive >= thresh).astype(int)
    if len(np.unique(y_pred_thresh)) > 1:  # Evita divisão por zero
        prec = precision_score(y_test, y_pred_thresh)
        rec = recall_score(y_test, y_pred_thresh)
        f1_thresh = f1_score(y_test, y_pred_thresh)
    else:
        prec = 0 if thresh > 0.5 else 1
        rec = 0 if thresh > 0.5 else 1
        f1_thresh = 0 if thresh > 0.5 else 1
    
    precision_list.append(prec)
    recall_list.append(rec)
    f1_list.append(f1_thresh)

plt.plot(thresholds_analysis, precision_list, label='Precisão', linewidth=2)
plt.plot(thresholds_analysis, recall_list, label='Recall', linewidth=2)
plt.plot(thresholds_analysis, f1_list, label='F1-Score', linewidth=2)
plt.axvline(x=threshold, color='red', linestyle='--', alpha=0.7)
plt.xlabel('Threshold')
plt.ylabel('Métrica')
plt.title('Métricas vs Threshold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nINTERPRETAÇÃO DA AUC:")
if auc_score >= 0.9:
    interpretation = "EXCELENTE"
elif auc_score >= 0.8:
    interpretation = "BOM"
elif auc_score >= 0.7:
    interpretation = "RAZOÁVEL"
elif auc_score >= 0.6:
    interpretation = "FRACO"
else:
    interpretation = "MUITO FRACO"

print(f"AUC = {auc_score:.3f} - Desempenho: {interpretation}")
print(f"\nSignificado:")
print(f"• AUC = 1.0: Classificador perfeito")
print(f"• AUC = 0.5: Classificador aleatório") 
print(f"• AUC > 0.7: Bom poder discriminativo")
print(f"• Nosso modelo: {auc_score:.1%} de chance de classificar corretamente um par aleatório")

In [ ]:
# Curva Precision-Recall adicional
print("CURVA PRECISION-RECALL")

# Calculando a curva Precision-Recall
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_proba_positive)
pr_auc = auc(recall_curve, precision_curve)

plt.figure(figsize=(12, 5))

# Curva Precision-Recall
plt.subplot(1, 2, 1)
plt.plot(recall_curve, precision_curve, linewidth=2, 
         label=f'PR Curve (AUC = {pr_auc:.3f})')
plt.axhline(y=np.mean(y_test), color='red', linestyle='--', 
           label=f'Baseline = {np.mean(y_test):.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.grid(True, alpha=0.3)

# Comparação ROC vs PR
plt.subplot(1, 2, 2)
metrics_comparison = ['ROC AUC', 'PR AUC', 'Accuracy', 'F1-Score']
scores = [auc_score, pr_auc, accuracy_manual, f1]
colors = ['blue', 'green', 'orange', 'red']

bars = plt.bar(metrics_comparison, scores, color=colors, alpha=0.7)
plt.title('Comparação de Métricas de Avaliação')
plt.ylabel('Score')
plt.ylim(0, 1)

for bar, score in zip(bars, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{score:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"PR AUC: {pr_auc:.3f}")
print(f"ROC AUC: {auc_score:.3f}")
print(f"\nQuando usar cada métrica:")
print(f"• ROC AUC: Melhor para datasets balanceados")
print(f"• PR AUC: Melhor para datasets desbalanceados com foco na classe positiva")

## 8. Análise de Resultados e Melhorias

Resumo final dos resultados e propostas de melhorias:

In [ ]:
# Resumo final dos resultados
print("RESUMO FINAL DOS RESULTADOS")
print("="*50)

# Informações do dataset
print(f"DATASET:")
print(f"• Total de registros: {len(df):,}")
print(f"• Features após encoding: {X_encoded.shape[1]}")
print(f"• Distribuição target: {target_counts[0]} não atrasados, {target_counts[1]} atrasados")

# Performance do modelo
print(f"\nPERFORMANCE DO MODELO:")
print(f"• Acurácia: {accuracy_manual:.1%}")
print(f"• Precisão: {precision:.1%}")
print(f"• Sensibilidade (Recall): {sensitivity_recall:.1%}")
print(f"• Especificidade: {specificity:.1%}")
print(f"• F1-Score: {f1:.3f}")
print(f"• ROC AUC: {auc_score:.3f}")
print(f"• PR AUC: {pr_auc:.3f}")

# Matriz de confusão resumo
print(f"\nMATRIZ DE CONFUSÃO:")
print(f"• Verdadeiros Positivos (TP): {tp}")
print(f"• Verdadeiros Negativos (TN): {tn}")
print(f"• Falsos Positivos (FP): {fp}")
print(f"• Falsos Negativos (FN): {fn}")

# Análise de negócio
print(f"\nANÁLISE DE NEGÓCIO:")
print(f"• Taxa de alarmes falsos: {fpr:.1%} (clientes alertados desnecessariamente)")
print(f"• Taxa de atrasos perdidos: {fnr:.1%} (atrasos não detectados)")

# Avaliação geral
print(f"\nAVALIAÇÃO GERAL:")
overall_score = (accuracy_manual + auc_score + f1) / 3
if overall_score >= 0.85:
    evaluation = "EXCELENTE"
elif overall_score >= 0.75:
    evaluation = "BOM"
elif overall_score >= 0.65:
    evaluation = "REGULAR"
else:
    evaluation = "PRECISA MELHORAR"

print(f"Score geral: {overall_score:.3f} - {evaluation}")

print(f"\nPROPOSTAS DE MELHORIAS:")
improvements = [
    "1. Otimização de hiperparâmetros (GridSearch/RandomSearch)",
    "2. Feature Engineering - criação de novas variáveis",
    "3. Tratamento de desbalanceamento (SMOTE, class_weight)",
    "4. Ensemble methods (Random Forest, Voting Classifier)",
    "5. Validação cruzada para estimativa mais robusta",
    "6. Análise de feature importance",
    "7. Otimização do threshold baseada no custo de negócio",
    "8. Coleta de mais dados ou features relevantes",
    "9. Análise temporal (sazonalidade, tendências)",
    "10. Implementação de modelos mais complexos (Neural Networks)"
]

for improvement in improvements:
    print(f"  {improvement}")

print(f"\nPRÓXIMOS PASSOS RECOMENDADOS:")
next_steps = [
    "• Implementar validação cruzada",
    "• Otimizar threshold baseado no contexto de negócio",
    "• Analisar feature importance do XGBoost",
    "• Testar outros algoritmos de classificação",
    "• Implementar pipeline de preprocessing completo"
]

for step in next_steps:
    print(f"  {step}")

print(f"\nATIVIDADE CONCLUÍDA COM SUCESSO!")
print("="*50)

## Conclusão da Atividade 2

### Objetivos Alcançados:

**1. Importar e preparar os dados**
- Dataset `flights_delays_120.csv` carregado e analisado
- Variáveis categóricas tratadas com `pd.get_dummies()`
- Divisão estratificada em treino e teste

**2. Treinar o modelo XGBoost**
- Classificador XGBoost configurado com parâmetros adequados
- Modelo treinado com sucesso

**3. Predição do modelo**
- Probabilidades obtidas com `predict_proba`
- Conversão para classes binárias com threshold 0.5
- Análise detalhada das probabilidades

**4. Matriz de Confusão**
- Matriz calculada e visualizada
- Extração de TP, TN, FP, FN
- Interpretação detalhada dos componentes

**5. Métricas de desempenho**
- Sensibilidade (Recall), Especificidade, FPR, FNR calculadas
- Acurácia, Precisão e F1-Score analisadas
- Comparação entre cálculos manuais e sklearn

**6. Curva ROC e AUC**
- Curva ROC gerada e visualizada
- AUC calculada e interpretada
- Curva Precision-Recall adicional
- Análise de diferentes thresholds

### Resultados Obtidos:
- **ROC AUC**: Métrica principal para avaliação do modelo
- **Visualizações completas**: Matriz de confusão, curvas ROC e PR
- **Métricas detalhadas**: Todas as métricas solicitadas calculadas
- **Interpretação de negócio**: Análise do impacto prático dos resultados

### Valor para o Negócio:
O modelo desenvolvido pode ser usado pelo site de reservas para:
- Informar probabilidade de atraso no momento da reserva
- Melhorar a experiência do cliente com informações precisas
- Reduzir reclamações através de expectativas adequadas

**Atividade 2 concluída com excelência!**